## **Import Libraries**

In [1]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
import random

In [2]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

random.seed(0)
weight_decay = 0.000005

In [3]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [4]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [5]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [6]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [7]:
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(X_train))
    X_test = normalize_minus1_1(cast_to_floatx(X_test))
    return (X_train, y_train), (X_test, y_test)

In [8]:
def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [9]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [10]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'cifar10': ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [11]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [12]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [13]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [14]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [15]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [16]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [17]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import fashion_mnist, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [18]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print(" Training Class ",single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    #Load the data
    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    #Transformations and Training the classifier
    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    print("START the proposed transformations.................. ")
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print("END the proposed transformations......................... ")
    print("START the training.................. ")

    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=BATCH_SIZE, epochs= N_EPOCHS 
            )
    print("END the training.................. ")
    
    #Estimating the parameters  
    def calc_approx_alpha_sum(observations):
        N = len(observations)
        f = np.mean(observations, axis=0)

        return (N * (len(f) - 1) * (-psi(1))) / (
                N * np.sum(f * np.log(f)) - np.sum(f * np.sum(np.log(observations), axis=0)) - np.sum(f * np.sum(np.log(1-observations), axis=0)) - np.sum([np.log(observations[j][k]/(1-observations[j][k])) for j in range(N)]))
    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x  

    def fixed_point_mul_beta(y,alpha_0,max_iter):
      N = y.shape[0]
      K = y.shape[1]
      y = np.clip(y,10**(-10),0.999)
      log_y = np.log(y)
      log_1_y = np.log(1-y)
      log_y_1_y = np.log(y/(1-y))
      f_y = np.mean(log_y, axis=0)
      f_1_y = np.mean(log_1_y, axis=0)
      f_y_1_y = np.mean(log_y_1_y, axis=0)
      alpha_old = alpha_0
      alpha_new = np.zeros(K)
      for iterr in range(max_iter):
          #calculate alpha
          for i in range(K):
            #aph = psi(np.sum(alpha_old)) + (1/N)*np.sum([np.log(y[j][i]) for j in range(N)]) - (1/N)*np.sum([np.log(1-y[j][i]) for j in range(N)]) - (1/N)*np.sum([np.log(y[j][i]/(1-y[j][i])) for j in range(N)])
            aph = psi(np.sum(alpha_old)) + f_y[i] - f_1_y[i] - (1/N)*np.sum([np.log(1 + np.sum([y[j][l]/(1-y[j][l]) for l in range(K)])) for j in range(N)])
            aph = np.nan_to_num(aph, copy=True, nan=-20, posinf=None, neginf=None)
            alpha_new[i] = inv_psi(np.clip(aph,None,20))
            alpha_new[i] = np.nan_to_num(alpha_new[i], copy=True, nan=-20, posinf=None, neginf=None)
            alpha_new[i] = np.clip(alpha_new[i],10**(-10),None)
          if np.sqrt(np.sum((alpha_old - alpha_new) ** 2)) < 10**(-7): 
            break
          #update the parameters
          alpha_old = alpha_new
      return alpha_new

    #Calculating the normality score
    def mul_beta_normality_score(alpha,y):
        N = y.shape[0]
        scores = np.asarray([np.sum((alpha-1)*np.log(y)[j]) - np.sum((alpha+1)*np.log(1-y)[j]) - np.sum(alpha)*np.log(1+np.sum(y[j]/(1-y[j]))) for j in range(N)]) 
        scores = np.nan_to_num(scores, copy=True, nan=10**7, posinf=10**7, neginf=-(10**7))
        return scores

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=BATCH_SIZE)
        
        y = observed_dirichlet # Our matrix of observations 
        alpha_0 = 0.05*np.ones(y.shape[1])
        mle_alpha_t = fixed_point_mul_beta(y,alpha_0,max_iter=10)
        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=BATCH_SIZE)
        x_test_p = np.nan_to_num(x_test_p, copy=True, nan=0.5, posinf=None, neginf=None)
        scores += mul_beta_normality_score(mle_alpha_t, x_test_p)

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [19]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    for i in range(n_runs):
        print('Run number:', i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]
        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [20]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 128
N_EPOCHS = 5

In [21]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        (load_fashion_mnist, 'fashion_mnist', 10),
        #(load_cifar10, 'cifar10',10)
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
 Training Class  0
Wide Residual Network-16-8 created.
START the proposed transformations.................. 
END the proposed transformations......................... 
START the training.................. 
Epoch 1/5


204/530 [==========>...................] - ETA: 34:13 - loss: 4.3209 - acc: 0.02 - ETA: 21:34 - loss: 4.3189 - acc: 0.02 - ETA: 20:40 - loss: 4.3162 - acc: 0.03 - ETA: 20:32 - loss: 4.3136 - acc: 0.04 - ETA: 20:32 - loss: 4.3089 - acc: 0.05 - ETA: 20:11 - loss: 4.3058 - acc: 0.05 - ETA: 20:22 - loss: 4.3022 - acc: 0.06 - ETA: 20:05 - loss: 4.2991 - acc: 0.06 - ETA: 19:53 - loss: 4.2943 - acc: 0.06 - ETA: 20:01 - loss: 4.2896 - acc: 0.06 - ETA: 19:55 - loss: 4.2848 - acc: 0.07 - ETA: 19:45 - loss: 4.2799 - acc: 0.07 - ETA: 19:40 - loss: 4.2732 - acc: 0.08 - ETA: 19:39 - loss: 4.2676 - acc: 0.08 - ETA: 19:37 - loss: 4.2608 - acc: 0.08 - ETA: 19:29 - loss: 4.2539 - acc: 0.08 - ETA: 19:24 - loss: 4.2460 - acc: 0.09 - ETA: 19:27 - loss: 4.2386 - acc: 0.09 - ETA: 19:23 - loss: 4.2310 - acc: 0.09 - ETA: 19:16 - loss: 4.2237 - acc: 0.09 - ETA: 19:13 - loss: 4.2156 - acc: 0.09 - ETA: 19:06 - loss: 4.2080 - acc: 0.09 - ETA: 19:05 - loss: 4.2002 - acc: 0.09 - ETA: 19:02 - loss: 4.1916 - acc: 0.10

408/530 [======================>.......] - ETA: 11:56 - loss: 1.9544 - acc: 0.39 - ETA: 11:54 - loss: 1.9488 - acc: 0.39 - ETA: 11:52 - loss: 1.9434 - acc: 0.39 - ETA: 11:49 - loss: 1.9382 - acc: 0.39 - ETA: 11:47 - loss: 1.9329 - acc: 0.40 - ETA: 11:45 - loss: 1.9283 - acc: 0.40 - ETA: 11:43 - loss: 1.9229 - acc: 0.40 - ETA: 11:41 - loss: 1.9176 - acc: 0.40 - ETA: 11:38 - loss: 1.9127 - acc: 0.40 - ETA: 11:36 - loss: 1.9078 - acc: 0.40 - ETA: 11:34 - loss: 1.9027 - acc: 0.40 - ETA: 11:32 - loss: 1.8977 - acc: 0.40 - ETA: 11:30 - loss: 1.8929 - acc: 0.40 - ETA: 11:27 - loss: 1.8882 - acc: 0.40 - ETA: 11:25 - loss: 1.8836 - acc: 0.40 - ETA: 11:23 - loss: 1.8784 - acc: 0.41 - ETA: 11:21 - loss: 1.8738 - acc: 0.41 - ETA: 11:19 - loss: 1.8689 - acc: 0.41 - ETA: 11:16 - loss: 1.8645 - acc: 0.41 - ETA: 11:14 - loss: 1.8599 - acc: 0.41 - ETA: 11:12 - loss: 1.8556 - acc: 0.41 - ETA: 11:10 - loss: 1.8509 - acc: 0.41 - ETA: 11:08 - loss: 1.8463 - acc: 0.41 - ETA: 11:06 - loss: 1.8419 - acc: 0.41

530/530 [==============================] - ETA: 4:26 - loss: 1.3364 - acc: 0.539 - ETA: 4:24 - loss: 1.3346 - acc: 0.539 - ETA: 4:22 - loss: 1.3331 - acc: 0.540 - ETA: 4:19 - loss: 1.3312 - acc: 0.540 - ETA: 4:17 - loss: 1.3294 - acc: 0.541 - ETA: 4:15 - loss: 1.3277 - acc: 0.541 - ETA: 4:13 - loss: 1.3260 - acc: 0.542 - ETA: 4:11 - loss: 1.3242 - acc: 0.542 - ETA: 4:08 - loss: 1.3226 - acc: 0.543 - ETA: 4:06 - loss: 1.3208 - acc: 0.543 - ETA: 4:04 - loss: 1.3192 - acc: 0.544 - ETA: 4:02 - loss: 1.3174 - acc: 0.544 - ETA: 3:59 - loss: 1.3157 - acc: 0.545 - ETA: 3:57 - loss: 1.3140 - acc: 0.545 - ETA: 3:55 - loss: 1.3124 - acc: 0.545 - ETA: 3:53 - loss: 1.3105 - acc: 0.546 - ETA: 3:51 - loss: 1.3089 - acc: 0.547 - ETA: 3:48 - loss: 1.3073 - acc: 0.547 - ETA: 3:46 - loss: 1.3055 - acc: 0.547 - ETA: 3:44 - loss: 1.3039 - acc: 0.548 - ETA: 3:42 - loss: 1.3021 - acc: 0.548 - ETA: 3:40 - loss: 1.3005 - acc: 0.549 - ETA: 3:37 - loss: 1.2988 - acc: 0.549 - ETA: 3:35 - loss: 1.2971 - acc: 0.550

204/530 [==========>...................] - ETA: 20:17 - loss: 0.5073 - acc: 0.81 - ETA: 19:40 - loss: 0.5486 - acc: 0.76 - ETA: 18:56 - loss: 0.5605 - acc: 0.76 - ETA: 18:54 - loss: 0.5510 - acc: 0.76 - ETA: 19:03 - loss: 0.5511 - acc: 0.76 - ETA: 19:11 - loss: 0.5507 - acc: 0.76 - ETA: 19:11 - loss: 0.5642 - acc: 0.75 - ETA: 19:04 - loss: 0.5527 - acc: 0.76 - ETA: 19:06 - loss: 0.5509 - acc: 0.76 - ETA: 19:10 - loss: 0.5549 - acc: 0.76 - ETA: 19:06 - loss: 0.5631 - acc: 0.75 - ETA: 19:03 - loss: 0.5603 - acc: 0.75 - ETA: 18:57 - loss: 0.5601 - acc: 0.75 - ETA: 18:59 - loss: 0.5608 - acc: 0.75 - ETA: 18:55 - loss: 0.5533 - acc: 0.76 - ETA: 18:53 - loss: 0.5542 - acc: 0.76 - ETA: 18:47 - loss: 0.5594 - acc: 0.76 - ETA: 18:45 - loss: 0.5559 - acc: 0.76 - ETA: 18:45 - loss: 0.5525 - acc: 0.76 - ETA: 18:43 - loss: 0.5476 - acc: 0.77 - ETA: 18:40 - loss: 0.5463 - acc: 0.76 - ETA: 18:36 - loss: 0.5480 - acc: 0.76 - ETA: 18:38 - loss: 0.5492 - acc: 0.76 - ETA: 18:34 - loss: 0.5483 - acc: 0.76

...

204/563 [=========>....................] - ETA: 19:28 - loss: 0.0345 - acc: 1.00 - ETA: 19:56 - loss: 0.0350 - acc: 1.00 - ETA: 19:37 - loss: 0.0350 - acc: 1.00 - ETA: 19:26 - loss: 0.0348 - acc: 1.00 - ETA: 19:15 - loss: 0.0347 - acc: 1.00 - ETA: 19:09 - loss: 0.0347 - acc: 1.00 - ETA: 19:07 - loss: 0.0346 - acc: 1.00 - ETA: 19:10 - loss: 0.0347 - acc: 1.00 - ETA: 19:05 - loss: 0.0346 - acc: 1.00 - ETA: 19:02 - loss: 0.0348 - acc: 1.00 - ETA: 19:00 - loss: 0.0347 - acc: 1.00 - ETA: 18:57 - loss: 0.0346 - acc: 1.00 - ETA: 18:54 - loss: 0.0346 - acc: 1.00 - ETA: 18:51 - loss: 0.0346 - acc: 1.00 - ETA: 18:51 - loss: 0.0345 - acc: 1.00 - ETA: 18:49 - loss: 0.0345 - acc: 1.00 - ETA: 18:46 - loss: 0.0344 - acc: 1.00 - ETA: 18:45 - loss: 0.0344 - acc: 1.00 - ETA: 18:42 - loss: 0.0344 - acc: 1.00 - ETA: 18:43 - loss: 0.0344 - acc: 1.00 - ETA: 18:40 - loss: 0.0344 - acc: 1.00 - ETA: 18:37 - loss: 0.0343 - acc: 1.00 - ETA: 18:34 - loss: 0.0343 - acc: 1.00 - ETA: 18:33 - loss: 0.0343 - acc: 1.00

408/563 [====================>.........] - ETA: 12:20 - loss: 0.0451 - acc: 0.99 - ETA: 12:18 - loss: 0.0451 - acc: 0.99 - ETA: 12:16 - loss: 0.0451 - acc: 0.99 - ETA: 12:14 - loss: 0.0452 - acc: 0.99 - ETA: 12:12 - loss: 0.0452 - acc: 0.99 - ETA: 12:10 - loss: 0.0452 - acc: 0.99 - ETA: 12:08 - loss: 0.0452 - acc: 0.99 - ETA: 12:06 - loss: 0.0452 - acc: 0.99 - ETA: 12:04 - loss: 0.0452 - acc: 0.99 - ETA: 12:02 - loss: 0.0452 - acc: 0.99 - ETA: 12:00 - loss: 0.0452 - acc: 0.99 - ETA: 11:58 - loss: 0.0452 - acc: 0.99 - ETA: 11:56 - loss: 0.0451 - acc: 0.99 - ETA: 11:53 - loss: 0.0451 - acc: 0.99 - ETA: 11:51 - loss: 0.0451 - acc: 0.99 - ETA: 11:49 - loss: 0.0451 - acc: 0.99 - ETA: 11:47 - loss: 0.0451 - acc: 0.99 - ETA: 11:45 - loss: 0.0451 - acc: 0.99 - ETA: 11:43 - loss: 0.0451 - acc: 0.99 - ETA: 11:41 - loss: 0.0451 - acc: 0.99 - ETA: 11:39 - loss: 0.0451 - acc: 0.99 - ETA: 11:37 - loss: 0.0450 - acc: 0.99 - ETA: 11:35 - loss: 0.0450 - acc: 0.99 - ETA: 11:33 - loss: 0.0451 - acc: 0.99

563/563 [==============================] - ETA: 5:18 - loss: 0.0423 - acc: 0.998 - ETA: 5:16 - loss: 0.0423 - acc: 0.998 - ETA: 5:14 - loss: 0.0424 - acc: 0.998 - ETA: 5:12 - loss: 0.0423 - acc: 0.998 - ETA: 5:10 - loss: 0.0423 - acc: 0.998 - ETA: 5:08 - loss: 0.0423 - acc: 0.998 - ETA: 5:06 - loss: 0.0423 - acc: 0.998 - ETA: 5:04 - loss: 0.0423 - acc: 0.998 - ETA: 5:02 - loss: 0.0423 - acc: 0.998 - ETA: 5:00 - loss: 0.0423 - acc: 0.998 - ETA: 4:58 - loss: 0.0423 - acc: 0.998 - ETA: 4:56 - loss: 0.0423 - acc: 0.998 - ETA: 4:53 - loss: 0.0422 - acc: 0.998 - ETA: 4:51 - loss: 0.0422 - acc: 0.998 - ETA: 4:49 - loss: 0.0423 - acc: 0.998 - ETA: 4:47 - loss: 0.0422 - acc: 0.998 - ETA: 4:45 - loss: 0.0422 - acc: 0.998 - ETA: 4:43 - loss: 0.0422 - acc: 0.998 - ETA: 4:41 - loss: 0.0422 - acc: 0.998 - ETA: 4:39 - loss: 0.0422 - acc: 0.998 - ETA: 4:37 - loss: 0.0422 - acc: 0.998 - ETA: 4:35 - loss: 0.0422 - acc: 0.998 - ETA: 4:33 - loss: 0.0422 - acc: 0.998 - ETA: 4:31 - loss: 0.0422 - acc: 0.998

/home/oussama/anaconda3/envs/oussama/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in log
/home/oussama/anaconda3/envs/oussama/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in true_divide
/home/oussama/anaconda3/envs/oussama/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


roc_auc 0.9954870601919162
pr_auc_norm where normal is the positive class 0.958300653425355
pr_auc_norm where anomaly is the positive class 0.9995326260350742
END _transformations_experiment 
